# 3ain Model using for detection

## Importat note: Skip Steps 1,2 if you did run (Train.ipynb)

### Step1 - Downloading yolov7 model with all files

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git

### Step2 - installing all required libraries for yolov7

In [ ]:
%cd yolov7
!pip install -r requirements.txt
!pip uninstall wandb -y

### Step3 - install tensorflow for CNN model

In [ ]:
try:
    !pip install tensorflow
except:
    pass
try:
    !pip uninstall h5py
except:
    pass
try:
    !pip install h5py
except:
    pass

### Step4 - Customize the (detect.py) file to include the CNN model

In [ ]:
new_file = 'import argparse\nimport time\nfrom pathlib import Path\n\nimport cv2\nimport torch\nimport torch.backends.cudnn as cudnn\nfrom numpy import random\nimport tensorflow as tf\nfrom keras.models import load_model\nfrom models.experimental import attempt_load\nfrom utils.datasets import LoadStreams, LoadImages\nfrom utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \\\n    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path\nfrom utils.plots import plot_one_box\nfrom utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel\nimport numpy as np\nfrom PIL import Image\n\n\n\n\nmodelg = load_model(\'CNN_Model.h5\')\nclass_names = [\'0\', \'1\', \'2\', \'3\', \'4\']\ndef CNN(x, img):\n    try:\n        cropped_image = img[int(x[1])-20:int(x[3])+20 , int(x[0])-20:int(x[2])+20]\n        color_converted = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)\n        pil_image=Image.fromarray(color_converted)\n        pil_image = pil_image.resize((224, 224))\n        img_array1 = tf.keras.utils.img_to_array(pil_image)\n        img_array = tf.expand_dims(img_array1, 0)\n        predictions = modelg.predict(img_array)\n        score = tf.nn.softmax(predictions[0])\n        result = class_names[np.argmax(score)]\n        return int(result)\n    except:\n        pass\n\n\ndef detect(save_img=False):\n    source, weights, view_img, save_txt, imgsz, trace = opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size, not opt.no_trace\n    save_img = not opt.nosave and not source.endswith(\'.txt\')  # save inference images\n    webcam = source.isnumeric() or source.endswith(\'.txt\') or source.lower().startswith(\n        (\'rtsp://\', \'rtmp://\', \'http://\', \'https://\'))\n\n    # Directories\n    save_dir = Path(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok))  # increment run\n    (save_dir / \'labels\' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir\n\n    # Initialize\n    set_logging()\n    device = select_device(opt.device)\n    half = device.type != \'cpu\'  # half precision only supported on CUDA\n\n    # Load model\n    model = attempt_load(weights, map_location=device)  # load FP32 model\n    stride = int(model.stride.max())  # model stride\n    imgsz = check_img_size(imgsz, s=stride)  # check img_size\n\n    if trace:\n        model = TracedModel(model, device, opt.img_size)\n\n    if half:\n        model.half()  # to FP16\n\n    # Second-stage classifier\n    classify = False\n    if classify:\n        modelc = load_classifier(name=\'resnet101\', n=2)  # initialize\n        modelc.load_state_dict(torch.load(\'weights/resnet101.pt\', map_location=device)[\'model\']).to(device).eval()\n\n    # Set Dataloader\n    vid_path, vid_writer = None, None\n    if webcam:\n        view_img = check_imshow()\n        cudnn.benchmark = True  # set True to speed up constant image size inference\n        dataset = LoadStreams(source, img_size=imgsz, stride=stride)\n    else:\n        dataset = LoadImages(source, img_size=imgsz, stride=stride)\n\n    # Get names and colors\n    names = model.module.names if hasattr(model, \'module\') else model.names\n    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]\n\n    # Run inference\n    if device.type != \'cpu\':\n        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once\n    old_img_w = old_img_h = imgsz\n    old_img_b = 1\n\n    t0 = time.time()\n    for path, img, im0s, vid_cap in dataset:\n        img = torch.from_numpy(img).to(device)\n        img = img.half() if half else img.float()  # uint8 to fp16/32\n        img /= 255.0  # 0 - 255 to 0.0 - 1.0\n        if img.ndimension() == 3:\n            img = img.unsqueeze(0)\n\n        # Warmup\n        if device.type != \'cpu\' and (old_img_b != img.shape[0] or old_img_h != img.shape[2] or old_img_w != img.shape[3]):\n            old_img_b = img.shape[0]\n            old_img_h = img.shape[2]\n            old_img_w = img.shape[3]\n            for i in range(3):\n                model(img, augment=opt.augment)[0]\n\n        # Inference\n        t1 = time_synchronized()\n        with torch.no_grad():   # Calculating gradients would cause a GPU memory leak\n            pred = model(img, augment=opt.augment)[0]\n        t2 = time_synchronized()\n\n        # Apply NMS\n        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)\n        t3 = time_synchronized()\n\n        # Apply Classifier\n        if classify:\n            pred = apply_classifier(pred, modelc, img, im0s)\n\n        # Process detections\n        for i, det in enumerate(pred):  # detections per image\n            if webcam:  # batch_size >= 1\n                p, s, im0, frame = path[i], \'%g: \' % i, im0s[i].copy(), dataset.count\n            else:\n                p, s, im0, frame = path, \'\', im0s, getattr(dataset, \'frame\', 0)\n\n            p = Path(p)  # to Path\n            save_path = str(save_dir / p.name)  # img.jpg\n            txt_path = str(save_dir / \'labels\' / p.stem) + (\'\' if dataset.mode == \'image\' else f\'_{frame}\')  # img.txt\n            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh\n            if len(det):\n                # Rescale boxes from img_size to im0 size\n                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()\n\n                # Print results\n                for c in det[:, -1].unique():\n                    n = (det[:, -1] == c).sum()  # detections per class\n                    s += f"{n} {names[int(c)]}{\'s\' * (n > 1)}, "  # add to string\n\n                # Write results\n                for *xyxy, conf, cls in reversed(det):\n                    if save_txt:  # Write to file\n                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh\n                        line = (cls, *xywh, conf) if opt.save_conf else (cls, *xywh)  # label format\n                        with open(txt_path + \'.txt\', \'a\') as f:\n                            f.write((\'%g \' * len(line)).rstrip() % line + \'\\n\')\n\n                    if save_img or view_img:  # Add bbox to image\n                        Classi = CNN(xyxy,im0)\n                        if Classi == 0:\n                            pass\n                        elif Classi == 1:\n                            plot_one_box(xyxy, im0, label="Low", color=colors[int(cls)], line_thickness=2)\n                        elif Classi == 2:\n                            plot_one_box(xyxy, im0, label="Moderate", color=colors[int(cls)], line_thickness=2)\n                        elif Classi == 3:\n                            plot_one_box(xyxy, im0, label="High", color=colors[int(cls)], line_thickness=2)\n                        elif Classi == 4:\n                            plot_one_box(xyxy, im0, label="Dangerous", color=colors[int(cls)], line_thickness=2)\n            print(f\'{s}Done. ({(1E3 * (t2 - t1)):.1f}ms) Inference, ({(1E3 * (t3 - t2)):.1f}ms) NMS\')\n\n            # Stream results\n            if view_img:\n                cv2.imshow(str(p), im0)\n                cv2.waitKey(1)  # 1 millisecond\n\n            # Save results (image with detections)\n            if save_img:\n                if dataset.mode == \'image\':\n                    cv2.imwrite(save_path, im0)\n                    print(f" The image with the result is saved in: {save_path}")\n                else:  # \'video\' or \'stream\'\n                    if vid_path != save_path:  # new video\n                        vid_path = save_path\n                        if isinstance(vid_writer, cv2.VideoWriter):\n                            vid_writer.release()  # release previous video writer\n                        if vid_cap:  # video\n                            fps = vid_cap.get(cv2.CAP_PROP_FPS)\n                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))\n                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))\n                        else:  # stream\n                            fps, w, h = 30, im0.shape[1], im0.shape[0]\n                            save_path += \'.mp4\'\n                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*\'mp4v\'), fps, (w, h))\n                    vid_writer.write(im0)\n\n    if save_txt or save_img:\n        s = f"\\n{len(list(save_dir.glob(\'labels/*.txt\')))} labels saved to {save_dir / \'labels\'}" if save_txt else \'\'\n        #print(f"Results saved to {save_dir}{s}")\n\n    print(f\'Done. ({time.time() - t0:.3f}s)\')\n\n\nif __name__ == \'__main__\':\n    parser = argparse.ArgumentParser()\n    parser.add_argument(\'--weights\', nargs=\'+\', type=str, default=\'yolov7.pt\', help=\'model.pt path(s)\')\n    parser.add_argument(\'--source\', type=str, default=\'inference/images\', help=\'source\')  # file/folder, 0 for webcam\n    parser.add_argument(\'--img-size\', type=int, default=640, help=\'inference size (pixels)\')\n    parser.add_argument(\'--conf-thres\', type=float, default=0.25, help=\'object confidence threshold\')\n    parser.add_argument(\'--iou-thres\', type=float, default=0.45, help=\'IOU threshold for NMS\')\n    parser.add_argument(\'--device\', default=\'\', help=\'cuda device, i.e. 0 or 0,1,2,3 or cpu\')\n    parser.add_argument(\'--view-img\', action=\'store_true\', help=\'display results\')\n    parser.add_argument(\'--save-txt\', action=\'store_true\', help=\'save results to *.txt\')\n    parser.add_argument(\'--save-conf\', action=\'store_true\', help=\'save confidences in --save-txt labels\')\n    parser.add_argument(\'--nosave\', action=\'store_true\', help=\'do not save images/videos\')\n    parser.add_argument(\'--classes\', nargs=\'+\', type=int, help=\'filter by class: --class 0, or --class 0 2 3\')\n    parser.add_argument(\'--agnostic-nms\', action=\'store_true\', help=\'class-agnostic NMS\')\n    parser.add_argument(\'--augment\', action=\'store_true\', help=\'augmented inference\')\n    parser.add_argument(\'--update\', action=\'store_true\', help=\'update all models\')\n    parser.add_argument(\'--project\', default=\'runs/detect\', help=\'save results to project/name\')\n    parser.add_argument(\'--name\', default=\'exp\', help=\'save results to project/name\')\n    parser.add_argument(\'--exist-ok\', action=\'store_true\', help=\'existing project/name ok, do not increment\')\n    parser.add_argument(\'--no-trace\', action=\'store_true\', help=\'don`t trace model\')\n    opt = parser.parse_args()\n    print(opt)\n    #check_requirements(exclude=(\'pycocotools\', \'thop\'))\n\n    with torch.no_grad():\n        if opt.update:  # update all models (to fix SourceChangeWarning)\n            for opt.weights in [\'yolov7.pt\']:\n                detect()\n                strip_optimizer(opt.weights)\n        else:\n            detect()\n'
W = open("detect.py","w")
W.write(new_file)
W.close()

### Step5 - Change (FILENAME) with the target file to detect

In [ ]:
!python detect.py --weights ..\best.pt --conf 0.6 --img-size 640 --source FILENAME

## All runs will be saved under the file (runs\detect\*)